In [1]:
from ising.model import IsingModelND
import numpy as np
from ipywidgets import interact
from scipy import constants

%load_ext autoreload
%autoreload 2

In [25]:
beta = 8
temperature = 1 / (constants.Boltzmann * beta)
model = IsingModelND(3, temperature=temperature, size=8, spin=1)
model.plot_state()

states = []
for n in range(int(1e5)):
    model._run_mcmc_step()
    states.append(model.state.copy())

@interact(value=0, min=0, max=len(states))
def update(i=(0, len(states) - 1)):
    model.plot_state(states[i])

In [24]:
idxs = np.ndindex(*model.state.shape)
for idx in idxs:
    # print(idx)
    pass

model.state.ravel()
x, y, z = np.indices(model.state.shape)

z.ravel()

array([ 0,  1,  2, ..., 13, 14, 15])